# Function Calling with Claude 3 on Amazon Bedrock

Function calling with Claude on Amazon Bedrock is available with legacy tool use structure. An updated version of tool use with Claude will be available on Bedrock soon. Reference [tool-use for more information on this updated feature set](https://docs.anthropic.com/claude/docs/tool-use).

With function calling, we can provide Claude with descriptions of tools and functions it can use, Claude is able to intelligently decide based on user query when and how to use those tools to help answer questions and complete tasks. In its response, Claude will suggest the tools to use to complete a task(s) and the paramters that should be passed when calling those functions. The function and its parameters are typically output by Claude in a well-defined format so that the client can parse and run the associated functions. Reference [legacy function calling with Claude](https://docs.anthropic.com/claude/docs/legacy-tool-use#example-legacy-tool-use-prompt).
The function calling process involves several steps:

1. The user's query and the definitions of the available tools and functions are provided to Claude as part of a single prompt.
2. Claude analyzes the user's query and determines which tool(s) or function(s) should be called and with what parameters.
3. Claude constructs a properly formatted function call following a specific syntax.
4. The client-side code intercepts this function call using a designated stop sequence and executes the actual function.
5. The result of the function execution is passed back to Claude.
6. Claude incorporates the function result to formulate a final response to the user's query.


Let's Look at Weather Check use cases.

### SetUp

In [ ]:
!pip install defusedxml
!pip install requests
!pip install boto3

In [2]:
# import the libraries
import sys
from defusedxml import ElementTree
from collections import defaultdict
import os
from typing import Any
import tools
import boto3
import json
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Define some helper functions

In [4]:
# function to create prompt
def create_prompt(tools_string, user_input):
    message_list = [
    {
        "role": 'user',
        "content": [ {"type": "text", "text":
            f"""In this environment you have access to a set of tools you can use to answer the user's question.

            You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
            <function_calls>
            <invoke>
            <tool_name>$TOOL_NAME</tool_name>
            <parameters>
            <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
            ...
            </parameters>
            </invoke>
            </function_calls>

            Here are the tools available:
            <tools>
            {tools_string}
            </tools>

            User query: {user_input}
            """}
        ]
    }
]

    return message_list


In [5]:
# function to get tool list
def add_tools():
    tools_string = ""
    for tool_spec in tools.list_of_tools_specs:
        tools_string += tool_spec
    return tools_string

In [6]:
# Function calling
def call_function(tool_name, parameters):
    func = getattr(tools, tool_name)
    output = func(**parameters)
    return output

In [7]:
# Format the functions results for input back in to Claude
def format_result(tool_name, output):
    return f"""
<function_results>
<result>
<tool_name>{tool_name}</tool_name>
<stdout>
{output}
</stdout>
</result>
</function_results>
"""

In [8]:
def etree_to_dict(t) -> dict[str, Any]:
    d = {t.tag: {}}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k: v[0] if len(v) == 1 else v for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(("@" + k, v) for k, v in t.attrib.items())
    if t.text and t.text.strip():
        if children or t.attrib:
            d[t.tag]["#text"] = t.text
        else:
            d[t.tag] = t.text
    return d

### Function Calling Prompt

In [9]:
def run_loop(prompt_data):
    # Start function calling loop
    while True:

        # initialize variables to make bedrock api call
        bedrock = boto3.client(service_name='bedrock-runtime')
        modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
        body=json.dumps(
                    {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1024,
            "messages": prompt_data,
            "temperature": 0,
            "top_p": 1,
            "system": '',
            "stop_sequences":["</function_calls>"]
            }  
            )  

        partial_completion = bedrock.invoke_model(
        body=body, 
        )
        
        response_body = json.loads(partial_completion.get('body').read())

        partial_completion= response_body.get('content')[0]['text']
        stop_reason=response_body.get('stop_reason')
        stop_seq = partial_completion.rstrip().endswith("</invoke>")
        
        # Get a completion from Claude
        # Append the completion to the end of the prommpt
        prompt_data.append({
                "role": 'assistant',
                "content": [
                    {"type": "text", "text": partial_completion}
                ]})
        
        if stop_reason == 'stop_sequence' and stop_seq:
            # If Claude made a function call
            start_index = partial_completion.find("<function_calls>")
            if start_index != -1:
                # Extract the XML Claude outputted (invoking the function)
                extracted_text = partial_completion[start_index+16:]
                print(extracted_text)
                # Parse the XML find the tool name and the parameters that we need to pass to the tool
                xml = ElementTree.fromstring(extracted_text)
                tool_name_element = xml.find("tool_name")
                if tool_name_element is None:
                    print("Unable to parse function call, invalid XML or missing 'tool_name' tag")
                    break
                tool_name_from_xml = tool_name_element.text.strip()
                parameters_xml = xml.find("parameters")
                if parameters_xml is None:
                    print("Unable to parse function call, invalid XML or missing 'parameters' tag")
                    break
                param_dict = etree_to_dict(parameters_xml)
                parameters = param_dict["parameters"]

                # Call the tool we defined in tools.py
                output = call_function(tool_name_from_xml, parameters)
                
                print("#################OUTPUT########################")
                print(output)
                print("###############################################")

                # Add the stop sequence back to the prompt
                # Add the result from calling the tool back to the prompt
                function_result = format_result(tool_name_from_xml, output)
                # print(function_result)
                prompt_data.append({
                    "role": 'user',
                    "content": [
                        {"type": "text", "text":f"""</function_calls> This is the result of the function{function_result}"""}
                    ]})

                # prompt_data += str(function_result)
        else:
            # If Claude did not make a function call
            # outputted answer
            print(partial_completion)
            break

In [10]:
import time
start_time = time.time()

user_input = "Can you check the weather for me in Paris, France?"
tools_string = add_tools()
prompt = create_prompt(tools_string, user_input)
run_loop(prompt)


print("--- %s seconds ---" % (time.time() - start_time))



<invoke>
<tool_name>get_lat_long</tool_name>
<parameters>
<place>Paris, France</place>
</parameters>
</invoke>

#################OUTPUT########################
{'latitude': '48.8534951', 'longitude': '2.3483915'}
###############################################

<invoke>
<tool_name>get_weather</tool_name>
<parameters>
<latitude>48.8534951</latitude>
<longitude>2.3483915</longitude>
</parameters>
</invoke>

#################OUTPUT########################
{'latitude': 48.86, 'longitude': 2.3399997, 'generationtime_ms': 0.04291534423828125, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 43.0, 'current_weather_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature': '°C', 'windspeed': 'km/h', 'winddirection': '°', 'is_day': '', 'weathercode': 'wmo code'}, 'current_weather': {'time': '2024-04-30T20:15', 'interval': 900, 'temperature': 15.2, 'windspeed': 7.4, 'winddirection': 133, 'is_day': 0, 'weathercode': 61}}
############################